In [57]:
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import shape
import re

# Allocating Suburbs to Local City Councils

The suburbs would be allocated to a local city council where the majority of the areas overlap

In [3]:
suburb = gpd.read_file("../data/raw/Suburb Shapes/vic_localities.shp")
council = gpd.read_file("../data/raw/VMADMIN/AD_LGA_AREA_POLYGON.shp")
council = council[council["STATE"]=="VIC"]

In [4]:
council['geometry'] = council['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
suburb['geometry'] = suburb['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# Finding overlaps

In [5]:
# Loop through each suburb and council
allocation = {}
for index, sub in suburb.iterrows():
    sub_geom = shape(sub["geometry"])
    max_area = 0
    council_name = None
    # If the council polygon contains the suburb polygon, then the suburb is assigned to the council
    # else the suburb will be assigned to the council that contains the most proportion of the suburb. 
    for index, coun in council.iterrows():
        con_geom = shape(coun["geometry"])
        if con_geom.contains(sub_geom):
            council_name  = coun["NAME"]
            break
        elif sub_geom.intersects(con_geom):
            area = sub_geom.intersection(con_geom).area/sub_geom.area
            if area > max_area:
                max_area = area
                council_name = coun["NAME"] 
    allocation[sub["LOC_NAME"]] = council_name

            



## Re-assigning crime rate data to each suburb 

In [64]:
# For each year, allocate each of the crime data by local gov council to the suburbs and save it as csv file
yrs = [f"{year}" for year in range(2013, 2023)]
wanted_rows = ["Suburb", "Local Government Area", "Year ending", "Alleged Offender Incident Rate", "Alleged Offender Incidents", "Criminial Incident Rate", "Family Incident Count", "Family Incident Rate", "Incidents Recorded", "Offence Count", "Offence Rate", "Victim Reports", "Victimisation Rate"]
for yr in yrs:
    suburb_crime = pd.DataFrame()
    suburb_crime["Suburb"] = allocation.keys()
    suburb_crime["Local Government Area"] = allocation.values()
    council_crime = pd.read_csv(f"../data/raw/Crime rates/{yr}_CrimeRates.csv")
    council_crime["Local Government Area"] = council_crime["Local Government Area"].apply(lambda x: re.sub("^ ", "", x).upper())
    result = suburb_crime.merge(council_crime, how="inner")
    result = result[wanted_rows]
    result.to_csv(f"../data/curated/{yr}_CrimeRate_Suburb.csv", index=False)
